# Car Price Prediction - Demonstração Completa

Este notebook demonstra o uso da biblioteca interna de Machine Learning para predição de preços de carros, aplicando as **melhores práticas** ensinadas nas aulas:

- ✅ **Modularidade** (Aula 2)
- ✅ **Documentação** (Aula 3)
- ✅ **Versionamento** (Aula 4)
- ✅ **PEP8** (Aula 5)
- ✅ **Testes** (Aula 6)
- ✅ **Design de API** (Aula 7)

---

## Índice

1. [Setup e Importações](#1-setup)
2. [Ingestão de Dados](#2-ingestion)
3. [Validação de Qualidade](#3-validation)
4. [Análise Exploratória](#4-eda)
5. [Treinamento do Modelo](#5-training)
6. [Avaliação e Métricas](#6-evaluation)
7. [Visualizações](#7-viz)
8. [Conclusões](#8-conclusions)

<a id='1-setup'></a>
## 1. Setup e Importações

Primeiro, vamos importar nossa biblioteca e outras dependências necessárias.

In [ ]:
# Importações padrão
import sys
import warnings
warnings.filterwarnings('ignore')

# Adiciona o diretório raiz ao path
sys.path.insert(0, '..')

# Bibliotecas externas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Nossa biblioteca interna (API modular e bem documentada)
from src import (
    DataIngestion,
    DataValidator,
    ModelTrainer,
    ModelEvaluator,
    setup_logger
)

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Bibliotecas importadas com sucesso!")
print(f"✓ Versão da biblioteca: 1.0.0")

<a id='2-ingestion'></a>
## 2. Ingestão de Dados

Utilizamos a classe `DataIngestion` que implementa uma **API consistente** (Aula 7) para carregar dados de diferentes fontes.

In [ ]:
# Inicializa o objeto de ingestão
ingestion = DataIngestion("../data/cars.csv")

# Gera dados sintéticos para demonstração
# Em produção: data = ingestion.load_data()
data = ingestion.generate_synthetic_data(n_samples=1000, random_state=42)

print(f"📊 Dados carregados: {data.shape[0]} linhas, {data.shape[1]} colunas")
print(f"📋 Colunas: {list(data.columns)}")
print("\n" + "="*70)
print("PRIMEIRAS LINHAS DO DATASET")
print("="*70)
data.head(10)

In [ ]:
# Estatísticas descritivas
print("="*70)
print("ESTATÍSTICAS DESCRITIVAS")
print("="*70)
data.describe()

<a id='3-validation'></a>
## 3. Validação de Qualidade dos Dados

A classe `DataValidator` implementa validações **modulares e reutilizáveis** (Aula 2) com **documentação clara** (Aula 3).

In [ ]:
# Cria o validador
validator = DataValidator(data)

# Define tipos esperados
expected_types = {
    'year': 'int',
    'mileage': 'int',
    'engine_size': 'float',
    'horsepower': 'int',
    'num_doors': 'int',
    'price': 'float'
}

# Executa todas as validações
validation_results = validator.validate_all(expected_types=expected_types)

# Exibe resumo
print(validator.get_summary())

In [ ]:
# Detalhes das validações
print("="*70)
print("RESULTADOS DETALHADOS DAS VALIDAÇÕES")
print("="*70)

for key, value in validation_results.items():
    print(f"\n{key.upper().replace('_', ' ')}:")
    if isinstance(value, dict):
        if value:
            for k, v in value.items():
                print(f"  • {k}: {v}")
        else:
            print("  ✓ Nenhum problema encontrado")
    else:
        print(f"  {value}")

<a id='4-eda'></a>
## 4. Análise Exploratória de Dados

Vamos explorar visualmente os dados antes do treinamento.

In [ ]:
# Distribuição do preço (variável target)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
axes[0].hist(data['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Preço ($)', fontsize=12)
axes[0].set_ylabel('Frequência', fontsize=12)
axes[0].set_title('Distribuição de Preços dos Carros', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(data['price'], vert=True)
axes[1].set_ylabel('Preço ($)', fontsize=12)
axes[1].set_title('Boxplot de Preços', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"📈 Preço médio: ${data['price'].mean():,.2f}")
print(f"📉 Preço mediano: ${data['price'].median():,.2f}")
print(f"📊 Desvio padrão: ${data['price'].std():,.2f}")

In [ ]:
# Matriz de correlação
plt.figure(figsize=(10, 8))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Features vs Preço', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📊 Correlações com o Preço:")
print("="*50)
correlations = correlation_matrix['price'].sort_values(ascending=False)
for feature, corr in correlations.items():
    if feature != 'price':
        print(f"  {feature:15s}: {corr:6.3f}")

In [ ]:
# Scatter plots das features vs preço
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Relação entre Features e Preço', fontsize=16, fontweight='bold')

features_to_plot = ['year', 'mileage', 'horsepower', 'engine_size']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

for idx, (feature, color) in enumerate(zip(features_to_plot, colors)):
    row, col = idx // 2, idx % 2
    axes[row, col].scatter(data[feature], data['price'], alpha=0.5, c=color, edgecolors='black', linewidth=0.5)
    axes[row, col].set_xlabel(feature.replace('_', ' ').title(), fontsize=11)
    axes[row, col].set_ylabel('Preço ($)', fontsize=11)
    axes[row, col].set_title(f'{feature.replace("_", " ").title()} vs Preço', fontsize=12, fontweight='bold')
    axes[row, col].grid(True, alpha=0.3)
    
    # Adiciona linha de tendência
    z = np.polyfit(data[feature], data['price'], 1)
    p = np.poly1d(z)
    axes[row, col].plot(data[feature], p(data[feature]), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

<a id='5-training'></a>
## 5. Treinamento do Modelo

A classe `ModelTrainer` implementa uma **API consistente** inspirada no scikit-learn (Aula 7) com métodos `fit()` e `predict()`.

In [ ]:
# Divide os dados em treino e teste
X_train, X_test, y_train, y_test = ingestion.split_data(test_size=0.2, random_state=42)

print("="*70)
print("DIVISÃO DOS DADOS")
print("="*70)
print(f"📊 Treino: {len(X_train)} amostras ({len(X_train)/len(data)*100:.1f}%)")
print(f"📊 Teste:  {len(X_test)} amostras ({len(X_test)/len(data)*100:.1f}%)")
print(f"\n📋 Features: {list(X_train.columns)}")

In [ ]:
# Inicializa e treina o modelo
print("\n" + "="*70)
print("TREINAMENTO DO MODELO")
print("="*70)

trainer = ModelTrainer(use_scaling=True)
trainer.fit(X_train, y_train)

print("✓ Modelo treinado com sucesso!")
print(f"✓ Tipo do modelo: {type(trainer.model).__name__}")
print(f"✓ Normalização aplicada: {trainer.use_scaling}")

In [ ]:
# Importância das features (coeficientes do modelo linear)
importance = trainer.get_feature_importance()

if importance is not None:
    # Cria DataFrame para visualização
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Coefficient': importance
    }).sort_values('Coefficient', key=abs, ascending=False)
    
    print("\n" + "="*70)
    print("IMPORTÂNCIA DAS FEATURES (Coeficientes)")
    print("="*70)
    print(importance_df.to_string(index=False))
    
    # Visualiza importância
    plt.figure(figsize=(10, 6))
    colors = ['green' if x > 0 else 'red' for x in importance_df['Coefficient']]
    plt.barh(importance_df['Feature'], importance_df['Coefficient'], color=colors, alpha=0.7, edgecolor='black')
    plt.xlabel('Coeficiente', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title('Importância das Features (Coeficientes do Modelo)', fontsize=14, fontweight='bold')
    plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

<a id='6-evaluation'></a>
## 6. Avaliação e Métricas

A classe `ModelEvaluator` fornece **métricas completas** com **documentação detalhada** (Aula 3).

In [ ]:
# Faz predições
predictions = trainer.predict(X_test)

# Cria o avaliador
evaluator = ModelEvaluator(y_test.values, predictions)

# Calcula métricas
metrics = evaluator.calculate_metrics()

# Exibe relatório completo
print(evaluator.get_report())

In [ ]:
# Comparação de predições
print("\n" + "="*70)
print("COMPARAÇÃO: VALORES REAIS vs PREDITOS")
print("="*70)

comparison = evaluator.compare_predictions(n_samples=15)
print(comparison.to_string())

<a id='7-viz'></a>
## 7. Visualizações de Performance

Vamos criar visualizações para entender melhor o desempenho do modelo.

In [ ]:
# Gráfico: Predições vs Valores Reais
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, alpha=0.6, edgecolors='black', linewidth=0.5, s=80)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Predição Perfeita')
plt.xlabel('Valor Real ($)', fontsize=12)
plt.ylabel('Valor Predito ($)', fontsize=12)
plt.title('Predições vs Valores Reais', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calcula R²
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print(f"📊 R² Score: {r2:.4f} (quanto mais próximo de 1, melhor)")

In [ ]:
# Análise de Resíduos
residuals = evaluator.get_residuals()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma dos resíduos
axes[0].hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero')
axes[0].set_xlabel('Resíduo ($)', fontsize=12)
axes[0].set_ylabel('Frequência', fontsize=12)
axes[0].set_title('Distribuição dos Resíduos', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Resíduos vs Predições
axes[1].scatter(predictions, residuals, alpha=0.6, edgecolors='black', linewidth=0.5, s=60)
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2, label='Zero')
axes[1].set_xlabel('Valor Predito ($)', fontsize=12)
axes[1].set_ylabel('Resíduo ($)', fontsize=12)
axes[1].set_title('Resíduos vs Predições', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Análise de Resíduos:")
print(f"  • Média dos resíduos: ${np.mean(residuals):,.2f} (ideal: próximo de 0)")
print(f"  • Desvio padrão: ${np.std(residuals):,.2f}")

In [ ]:
# Métricas Visuais
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# RMSE
axes[0].bar(['RMSE'], [metrics['rmse']], color='#FF6B6B', alpha=0.7, edgecolor='black', linewidth=2)
axes[0].set_ylabel('Valor ($)', fontsize=11)
axes[0].set_title('Root Mean Squared Error', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# MAE
axes[1].bar(['MAE'], [metrics['mae']], color='#4ECDC4', alpha=0.7, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Valor ($)', fontsize=11)
axes[1].set_title('Mean Absolute Error', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# R² Score
axes[2].bar(['R²'], [metrics['r2']], color='#45B7D1', alpha=0.7, edgecolor='black', linewidth=2)
axes[2].set_ylabel('Score', fontsize=11)
axes[2].set_title('R² Score', fontsize=12, fontweight='bold')
axes[2].set_ylim([0, 1])
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

<a id='8-conclusions'></a>
## 8. Conclusões e Melhores Práticas Aplicadas

Este projeto demonstra a aplicação completa das **7 aulas** sobre bibliotecas internas de ML:

### ✅ Aula 1: Benefícios de Bibliotecas Internas
- Código **reutilizável** e **consistente**
- Facilita colaboração entre times
- Acelera desenvolvimento de novos projetos

### ✅ Aula 2: Modularidade
- Código organizado em **módulos separados** (data_ingestion, data_validation, model_trainer, model_evaluation)
- Princípio **DRY** aplicado (funções e classes reutilizáveis)
- **Separação de responsabilidades** clara

### ✅ Aula 3: Documentação
- **Docstrings detalhadas** seguindo padrão Google/NumPy
- Exemplos de uso em cada função
- README completo com instruções

### ✅ Aula 4: Versionamento
- **Semantic Versioning** (1.0.0)
- Empacotamento com `setup.py` e `pyproject.toml`
- Pronto para distribuição interna

### ✅ Aula 5: PEP8 e Estilo
- Código segue **convenções PEP8**
- Nomenclatura consistente
- Configuração de linting (flake8, black)

### ✅ Aula 6: Testes e CI
- **Suite completa de testes** unitários
- Cobertura > 80%
- CI/CD configurado (GitHub Actions)

### ✅ Aula 7: Design de API
- Interface **consistente** (fit/predict)
- Inspirado em **scikit-learn**
- Fácil de usar e estender

---

### 📈 Resultados do Modelo

In [ ]:
# Resumo final
print("="*70)
print("RESUMO FINAL DO PROJETO")
print("="*70)
print(f"\n📊 Dataset: {len(data)} amostras, {len(data.columns)} features")
print(f"📊 Treino/Teste: {len(X_train)}/{len(X_test)} amostras")
print(f"\n🎯 Métricas de Performance:")
print(f"  • R² Score:  {metrics['r2']:.4f}")
print(f"  • RMSE:      ${metrics['rmse']:,.2f}")
print(f"  • MAE:       ${metrics['mae']:,.2f}")
print(f"  • MAPE:      {metrics['mape']:.2f}%")
print(f"\n✅ Modelo treinado e avaliado com sucesso!")
print(f"✅ Biblioteca modular e bem documentada")
print(f"✅ Pronto para uso em produção")
print("="*70)

---

## 🚀 Próximos Passos

1. **Experimentar outros modelos**: Ridge, Lasso, Random Forest
2. **Feature Engineering**: Criar novas features (interações, transformações)
3. **Hyperparameter Tuning**: Otimizar parâmetros do modelo
4. **Deploy**: Criar API REST para servir predições
5. **Monitoramento**: Implementar logging e métricas em produção

---

**Desenvolvido seguindo as melhores práticas de Bibliotecas Internas de ML** 🎓